<a href="https://colab.research.google.com/github/BlackeWhite/SSD_Modanet/blob/master/SSD300/Traning_ssd300_FineTuning07%2B%2B12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Google Drive File

In [ ]:
#per salvare file di gorsse dimensioni sul drive 
!pip install httplib2==0.15.0
!pip install google-api-python-client==1.6
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#VGG_VOC0712Plus_SSD_300x300_iter_240000
Weight = drive.CreateFile({'id': '1M99knPZ4DpY9tI60iZqxXsAxX2bYWDvZ'}) 
Weight.GetContentFile(Weight['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Weight['title'], Weight['id']))
#dataset
Modadat = drive.CreateFile({'id': '1il_eW1COHs1_QijMh3zklvhmE_Dfk0qh'}) 
Modadat.GetContentFile(Modadat['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Modadat['title'], Modadat['id']))
!unzip dataset.zip 
#annotazioni 
annotaz = drive.CreateFile({'id': '1aD4BgDD7grEFYlDKr0k_SHPw1qbt9N1q'}) 
annotaz.GetContentFile(annotaz['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(annotaz['title'], annotaz['id']))
!unzip annotazioni.zip 

# Import library

Import library from github of Pireluigi Ferrari that had created the code for implement the SSD

In [ ]:
!pip install keras==2.2.3

!pip install tensorflow-gpu==1.15

import h5py
import numpy as np
import shutil
import os.path
if not(os.path.isdir('keras_layers')):
  !git clone https://github.com/pierluigiferrari/ssd_keras.git
  !mv ./ssd_keras/* . #copy all file in the root directory, you must this because after invocate the lib in the repository 
  !rm -r ssd_keras

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation, SSDRandomCrop
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

import logging

%matplotlib inline

# Set the model configuration parameters

In [ ]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 120
steps_per_epoch = 256

img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 13 # Number of positive classes
scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets

aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True


# Build the model or load the model

Create a copy of 07++12 weights

In [ ]:
# TODO: Set the path for the source weights file you want to load.

weights_source_path = '/content/VGG_VOC0712Plus_SSD_300x300_iter_240000.h5'

# TODO: Set the path and name for the destination weights file
#       that you want to create.
weights_destination_path = '/content/VGG_VOC0712Plus_SSD_300x300_iter_240000_13classes.h5'

# Make a copy of the weights file.
shutil.copy(weights_source_path, weights_destination_path)

# Load both the source weights file and the copy we made.
# We will load the original weights file in read-only mode so that we can't mess up anything.
weights_source_file = h5py.File(weights_source_path, 'r')
weights_destination_file = h5py.File(weights_destination_path)


I define for which weights we have to modify the number of classes

In [ ]:
classifier_names = ['conv4_3_norm_mbox_conf',
                    'fc7_mbox_conf',
                    'conv6_2_mbox_conf',
                    'conv7_2_mbox_conf',
                    'conv8_2_mbox_conf',
                    'conv9_2_mbox_conf']

In [ ]:
conv4_3_norm_mbox_conf_kernel = weights_source_file[classifier_names[0]][classifier_names[0]]['kernel:0']
conv4_3_norm_mbox_conf_bias = weights_source_file[classifier_names[0]][classifier_names[0]]['bias:0']

print("Shape of the '{}' weights:".format(classifier_names[0]))
print()
print("kernel:\t", conv4_3_norm_mbox_conf_kernel.shape)
print("bias:\t", conv4_3_norm_mbox_conf_bias.shape)

Shape of the 'conv4_3_norm_mbox_conf' weights:

kernel:	 (3, 3, 512, 84)
bias:	 (84,)


The code in this section iterates over all the classifier layers of the source weights file and performs the following steps for each classifier layer:

1.   Get the kernel and bias tensors from the source weights file.
2.   Compute the sub-sampling indices for the last axis. The first three axes of the kernel remain unchanged.
3. Overwrite the corresponding kernel and bias tensors in the destination weights file with our newly created sub-sampled kernel and bias tensors.

In [ ]:
from misc_utils.tensor_sampling_utils import sample_tensors

In [ ]:
#       the background class, so for pascal voc's 20 classes, this must be 20 + 1 = 81.
n_classes_source = 21
# TODO: Set the indices of the classes that you want to pick for the sub-sampled weight tensors.
#       In case you would like to just randomly sample a certain number of classes, you can just set
#       `classes_of_interest` to an integer instead of the list below. Either way, don't forget to
#       include the background class. That is, if you set an integer, and you want `n` positive classes,
#       then you must set `classes_of_interest = n + 1`.
#classes_of_interest = [0, 3, 8, 1, 2, 10, 4, 6, 12]
classes_of_interest = 14

for name in classifier_names:
    # Get the trained weights for this layer from the source HDF5 weights file.
    kernel = weights_source_file[name][name]['kernel:0'].value
    bias = weights_source_file[name][name]['bias:0'].value

    # Get the shape of the kernel. We're interested in sub-sampling
    # the last dimension, 'o'.
    height, width, in_channels, out_channels = kernel.shape
    
    # Compute the indices of the elements we want to sub-sample.
    # Keep in mind that each classification predictor layer predicts multiple
    # bounding boxes for every spatial location, so we want to sub-sample
    # the relevant classes for each of these boxes.
    if isinstance(classes_of_interest, (list, tuple)):
        subsampling_indices = []
        for i in range(int(out_channels/n_classes_source)):
            indices = np.array(classes_of_interest) + i * n_classes_source
            subsampling_indices.append(indices)
        subsampling_indices = list(np.concatenate(subsampling_indices))
    elif isinstance(classes_of_interest, int):
        subsampling_indices = int(classes_of_interest * (out_channels/n_classes_source))
    else:
        raise ValueError("`classes_of_interest` must be either an integer or a list/tuple.")
    
    # Sub-sample the kernel and bias.
    # The `sample_tensors()` function used below provides extensive
    # documentation, so don't hesitate to read it if you want to know
    # what exactly is going on here.
    new_kernel, new_bias = sample_tensors(weights_list=[kernel, bias],
                                          sampling_instructions=[height, width, in_channels, subsampling_indices],
                                          axes=[[3]], # The one bias dimension corresponds to the last kernel dimension.
                                          init=['gaussian', 'zeros'],
                                          mean=0.0,
                                          stddev=0.005)
    
    # Delete the old weights from the destination file.
    del weights_destination_file[name][name]['kernel:0']
    del weights_destination_file[name][name]['bias:0']
    # Create new datasets for the sub-sampled weights.
    weights_destination_file[name][name].create_dataset(name='kernel:0', data=new_kernel)
    weights_destination_file[name][name].create_dataset(name='bias:0', data=new_bias)

# Make sure all data is written to our output file before this sub-routine exits.
weights_destination_file.flush()

# Inspect the shapes of the weights of the 'conv4_3_norm_mbox_conf' layer in the destination weights file
conv4_3_norm_mbox_conf_kernel = weights_destination_file[classifier_names[0]][classifier_names[0]]['kernel:0']
conv4_3_norm_mbox_conf_bias = weights_destination_file[classifier_names[0]][classifier_names[0]]['bias:0']

print("Shape of the '{}' weights:".format(classifier_names[0]))
print()
print("kernel:\t", conv4_3_norm_mbox_conf_kernel.shape)
print("bias:\t", conv4_3_norm_mbox_conf_bias.shape)

Shape of the 'conv4_3_norm_mbox_conf' weights:

kernel:	 (3, 3, 512, 56)
bias:	 (56,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


In [ ]:
K.clear_session() # Clear previous models from memory.

model, predictor_sizes = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=13,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels,
                return_predictor_sizes=True)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
#weights_path = '' #pesi all'epoca n
weights_path = weights_destination_path


model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=sgd, loss=ssd_loss.compute_loss,  metrics=["acc"])


# Load dataset

In [ ]:
def rename(path):   #funzione per apportare modifiche ai nomi del dataset 
	lis = os.listdir(path)
	for item in lis:
		temp=item.split(".")
		while len(temp[0])<7:
			new=str(0)+temp[0]
			temp[0]=new
			if len(temp[0])==7:
				os.rename(path+item,path+temp[0]+".jpg")

In [ ]:
rename('./new_train/')
rename('./new_val/')
rename('./new_test/')

In [ ]:
# TODO: Set the paths to your dataset here.
train_images_path = ['./new_train/']
train_labels_path = ['annotazioni_train.json']
val_images_path= ['./new_val/']
val_labels_path = ['annotazioni_val.json']

train_dataset = DataGenerator(load_images_into_memory=False)
val_dataset = DataGenerator(load_images_into_memory=False)

train_dataset.parse_json(images_dirs=train_images_path,
                  annotations_filenames=train_labels_path,
                  ground_truth_available= True,
                  include_classes = 'all',
                  ret=True)

val_dataset.parse_json(images_dirs=val_images_path,
                  annotations_filenames=val_labels_path,
                  ground_truth_available= True,
                  include_classes = 'all',
                  ret=True)


print("Number of images in the traning dataset:", train_dataset.get_dataset_size())
print("Number of images in the validation dataset:", val_dataset.get_dataset_size())

Processing 'annotazioni_val.json': 100%|██████████| 5225/5225 [00:00<00:00, 82671.56it/s]
Number of images in the traning dataset: 41810
Number of images in the validation dataset: 5225


In [ ]:
# 3: Set the batch size.

batch_size = 64

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)



ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[convert_to_3_channels,resize],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)
# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	 41810
Number of images in the validation dataset:	  5225


Set the remaining training parameters

In [ ]:
logging.debug('Training preparation')
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/Modelli_train/ssd300_07++12_minore_lr/ssd300_Modanet_07++12_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
#model_checkpoint.best = 

csv_logger = CSVLogger(filename='/content/drive/My Drive/Modelli_train/ssd300_07++12_minore_lr/log_ssd300_Modanet_07++12_.csv',
                       separator=',',
                       append=True)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
                                                verbose=1)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             learning_rate_scheduler,
             terminate_on_nan]

logging.debug('Training prepared')

# Train the model

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

# Generazione dei bounding box predetti e reali 

In [ ]:
y_pred = model.predict(batch_images)

y_pred_decoded = decode_detections(y_pred,
                                   confidence_thresh=0.5,
                                   iou_threshold=0.4,
                                   top_k=200,
                                   normalize_coords=normalize_coords,
                                   img_height=img_height,
                                   img_width=img_width)

In [ ]:
y_pred_decoded_inv = apply_inverse_transforms(y_pred_decoded, batch_inverse_transforms)

np.set_printoptions(precision=2, suppress=True, linewidth=90)
print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
print(y_pred_decoded_inv[i])

In [ ]:
colors = plt.cm.hsv(np.linspace(0, 1, n_classes+1)).tolist()

classes = ['background',
           'bag', 'belt', 'boots', 'footwear',
           'outer', 'dress', 'sunglasses', 'pants',
           'top', 'shorts', 'skirt', 'headwear',
           'scarf/tie']

plt.figure(figsize=(20,12))
plt.imshow(batch_original_images[i])

current_axis = plt.gca()

for box in batch_original_labels[i]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    label = '{}'.format(classes[int(box[0])])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})

for box in y_pred_decoded_inv[i]:
    xmin = box[2]
    ymin = box[3]
    xmax = box[4]
    ymax = box[5]
    color = colors[int(box[0])]
    label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})